# 区域卷积神经网络（R-CNN）

* 区域卷积神经网络（region-based CNN或regions with CNN features，R-CNN） 
* 快速的R-CNN（Fast R-CNN）
* 更快的R-CNN（Faster R-CNN）
* 掩码R-CNN（Mask R-CNN）

## R-CNN

* 阅读：区域卷积神经网络\_R-CNNs.pdf
* 每个提议区域都将被标注类别和真实边界框
* 将每个提议区域变形为网络需要的输入尺寸


### 训练

* 训练多个支持向量机对目标分类
* 训练线性回归模型来预测真实边界框

### 不足

* 速度很慢
* 可能从一张图像中选出上千个提议区域
* 上千次的卷积神经网络的前向传播

## Fast R-CNN

**回顾R-CNN**

* 对每个提议区域，卷积神经网络的前向传播是独立的，而没有共享计算
* 这些区域通常有重叠，独立的特征抽取会导致重复的计算

### region of interest (RoI) pooling layer

* 仅在整张图象上执行卷积神经网络的前向传播
* Suppose that selective search generates $n$ region proposals
* These region proposals (of different shapes) mark regions of interest (of different shapes) on the CNN output

* 输入图像：$40\times 40$
* 
```
X = torch.arange(16.).reshape(1, 1, 4, 4)
tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])
```
* 两个提议区域
```
rois = torch.Tensor(
    [[0, 0, 0, 20, 20], [0, 0, 10, 30, 30]])
```

* `spatial_scale=0.1`
* 两个兴趣区域`X[:, :, 0:3, 0:3]`和`X[:, :, 1:4, 0:4]`
* 这些感兴趣的区域需要进一步抽取出形状相同的特征（in order to be easily concatenated）

### RoI pooling layer

* 每个区域输出的高和宽分别为$h_2$和$w_2$
* 形状为$h \times w$的兴趣区域窗口
* 该窗口将被划分为$h_2 \times w_2$子窗口网格，其中每个子窗口的大小约为$\tfrac{h}{h_2} \times \tfrac{w}{w_2}$
* 最大元素作为该子窗口的输出
* 阅读：区域卷积神经网络\_R-CNNs.pdf

阅读实例：

```
torchvision.ops.roi_pool(X, 
    rois, output_size=(2, 2), spatial_scale=0.1)
```

## Faster R-CNN

阅读：区域卷积神经网络\_R-CNNs.pdf

### region proposal network

* 输入：an image (of any size)
* 输出：a set of rectangular object proposals, each with an objectness score.

<center><img src="../img/Region_Proposal_Network.png" width="40%"></center>

### “滑动窗口 + 锚框”之滑动窗口

1. slide a small network over the convolutional feature map

   $3\times 3$ spatial window
1. two sibling fully-connected layers（$1\times 1$卷积实现）
    * a box-regression layer
    * a box-classification layer

（结合上一slide中的figure）

### “滑动窗口 + 锚框”之锚框

* 最多$k$个proposals
* $k$个proposals：parameterized *relative* to $k$ reference boxes (*anchors*)
* 3 scales and 3 aspect ratios：$k = 9$ anchors
* 卷积特征图：$W\times H$
* 总共$W Hk$ anchors

### 训练RPN网络

\begin{equation*}
\begin{aligned}
t_{\textrm{x}} &=  (x - x_{\textrm{a}})/w_{\textrm{a}},\quad
t_{\textrm{y}} = (y - y_{\textrm{a}})/h_{\textrm{a}},\\
t_{\textrm{w}} &= \log(w / w_{\textrm{a}}), \quad
t_{\textrm{h}} = \log(h / h_{\textrm{a}}),\\
t^{*}_{\textrm{x}} &=  (x^{*} - x_{\textrm{a}})/w_{\textrm{a}},\quad
t^{*}_{\textrm{y}} = (y^{*} - y_{\textrm{a}})/h_{\textrm{a}},\\
t^{*}_{\textrm{w}} &= \log(w^{*} / w_{\textrm{a}}),\quad
t^{*}_{\textrm{h}} = \log(h^{*} / h_{\textrm{a}}),
\end{aligned}
\end{equation*}

* $x,y$：center的坐标；$w,h$：宽、高
* $x,x_a,x^*$：predicted box, anchor box, and ground-truth box

### Faster R-CNN总结

* 之前：
    * bounding-box regression is performed on features pooled from *arbitrarily* sized RoIs
    * the regression weights are *shared* by all region sizes




**thanks to the design of anchors**
 
* Faster R-CNN中：
    * the features used for regression are of the *same* spatial size ($3\times 3$) on the feature maps
    * varying sizes如何处理呢
    * $k$个bounding-box regressors
    * 每个regressor：responsible for one scale and one aspect ratio
    * regressors之间：do *not* share weights